In [1]:
import sys
import glob
import os
import ffmpeg
import audioread
from pydub import AudioSegment
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
import pickle

In [2]:
fma_path = 'C:/Users/ishmail.grady/Downloads/fma_small'
sys.path.append(fma_path)

In [3]:
metadata_path = 'C:/Users/ishmail.grady/Downloads/metadata'

genres_path = 'C:/Users/ishmail.grady/Downloads/metadata/genres.csv'
tracks_path = 'C:/Users/ishmail.grady/Downloads/metadata/tracks.csv'

sys.path.append(metadata_path)

In [4]:
genres = pd.read_csv(genres_path, index_col = 0)
tracks = pd.read_csv(tracks_path, header = [0,1], index_col = 0)

In [5]:
cols = [('set', 'split'), ('set', 'subset'), ('track', 'genre_top')]
tracks = tracks[cols]

In [6]:
def get_files(basedir, ext = '.mp3'):
    paths = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join( root, '*'+ext))
        paths.extend(files)
    return paths

In [7]:
def make_spectrogram(file_path):
    y , sr = librosa.load(file_path)
    spect = librosa.feature.melspectrogram( y = y, sr = sr, n_fft = 2048, hop_length = 512)
    spect = librosa.power_to_db(spect, ref = np.max)
    return spect.T

In [8]:
def plot_spectrogram(spect):
    plt.figure(figsize = (10,5))
    librosa.display.specshow(spect.T, y_axis = 'mel', fmax = 8000, x_axis = 'time')
    plt.colorbar( format = '%+2.0f dB')
    plt.show()
    

In [15]:
def create_array(paths):
    X_spect = np.empty((0,640,128))
    failed_tracks = []
    count = 0
    for file in paths:
        try:
            count += 1
            spect = make_spectrogram(file)
            spect = spect[:640, :]
            X_spect = np.append(X_spect, [spect], axis = 0)
            
            if count % 100 == 0:
                print('Currently processing: ', count)
        except:
            failed_tracks.append(file)
            print('Could Not Process: ', count)
            continue
    return X_spect , failed_tracks

In [16]:
track_paths = get_files(fma_path)

In [17]:
X_spect , failed_tracks  = create_array(track_paths)

Currently processing:  100
Currently processing:  200
Currently processing:  300
Currently processing:  400
Currently processing:  500
Currently processing:  600
Currently processing:  700
Currently processing:  800
Currently processing:  900
Currently processing:  1000
Currently processing:  1100
Currently processing:  1200
Currently processing:  1300
Currently processing:  1400
Currently processing:  1500
Currently processing:  1600
Currently processing:  1700
Currently processing:  1800
Currently processing:  1900
Currently processing:  2000
Currently processing:  2100
Currently processing:  2200
Currently processing:  2300
Currently processing:  2400
Currently processing:  2500
Currently processing:  2600
Currently processing:  2700
Currently processing:  2800
Currently processing:  2900
Currently processing:  3000
Currently processing:  3100
Currently processing:  3200
Currently processing:  3300
Currently processing:  3400
Currently processing:  3500
Currently processing:  3600
C

In [75]:
X_spect.shape

(7994, 640, 128)

In [76]:
X_spect_scaled = librosa.core.db_to_power(X_spect, ref = 1.0)

In [90]:
X_spect_log = np.log(X_spect_scaled)

In [91]:
np.savez('all_spects_scaled',X_spect_log)

In [28]:
tracks.columns = tracks.columns.droplevel(0)

In [31]:
tracks_small  = tracks[tracks.subset == 'small']

In [56]:
genres = []
splits = []
track_ids = [int(t[-10:-4]) for t in track_paths]
for t in track_ids:
    genres.append(tracks_small.loc[t, 'genre_top'])
    splits.append(tracks_small.loc[t, 'split'])

In [58]:
data = {'split':splits,'genre_top':genres}
track_info = pd.DataFrame(data, index = track_ids)

In [61]:
failed_track_ids = [int(t[-10:-4]) for t in failed_tracks]

In [65]:
track_info = track_info.drop(failed_track_ids)

In [71]:
track_info.to_pickle('track_info.pickle')